In [ ]:
import itertools
import re

from dataclasses import dataclass

In [ ]:
@dataclass
class Player:
    number: int
    position: int
    score: int = 0

In [ ]:
def get_players(filename):
    players = []
    with open(filename) as file:
        for line in file:
            number, position = re.findall("\d+", line)
            players.append(Player(number=int(number), position=int(position)))
    return players

# Part 1

In [ ]:
def play_until_wins(players, target=1000):
    dice = itertools.cycle(range(1, 101))
    rolls = 0

    while True:
        for player in players:
            throw = sum(next(dice) for _ in range(3))
            player.position = (player.position + throw - 1) % 10 + 1
            player.score += player.position
            rolls += 3
            if player.score >= target:
                solution = min(p.score for p in players)*rolls
                return player.number, solution

In [ ]:
players = get_players("day21.input")

winner, solution = play_until_wins(players)
print("Solution:", solution)

# Part 2

In [ ]:
import collections
from functools import cache

In [ ]:
# At each players turn, there is 27 possible outcomes for the sum of three throws.
# Since the sums 4-8 happens multiple times, let's compute the count of each sum
# and just multiply the simulation-outcome with that count.
possible_sums = collections.Counter(
    sum(dice) for dice in itertools.product([1, 2, 3], repeat=3)
)

#
# The positions can only be 1-10 and the scores can only be 0-21 (or slightly more).
# So the recursive function can only be called in 10*10*21*21 = 44100 ways.
# Thus, by caching the results we greatly speed up the calculation.
#
# This is acutally dynamic programming, but we're letting Python handle the storing.
# We could have created a dictionary with the tuple (p1_pos, p2_pos, p1_score, p2_score)
# as key, storing the result of that simulation.
#
@cache
def simulate_game(p1_pos, p2_pos, p1_score=0, p2_score=0):

    total_p1_wins, total_p2_wins = 0, 0

    if p1_score >= 21:
        return (1, 0)
    if p2_score >= 21:
        return (0, 1)

    # Take care not to change the p1_pos and p1_score, as it screws up the recursion...
    for dice_sum, sum_count in possible_sums.items():
        new_p1_pos = (p1_pos + dice_sum - 1) % 10 + 1
        new_p1_score = p1_score + new_p1_pos
        
        # Switch p1 and p2 to avoid writing the same logic twice.
        p2_wins, p1_wins = simulate_game(p2_pos, new_p1_pos, p2_score, new_p1_score)
        
        total_p1_wins += p1_wins * sum_count
        total_p2_wins += p2_wins * sum_count
    
    return total_p1_wins, total_p2_wins

In [ ]:
players = get_players("day21.input")

print("Solution:", max(simulate_game(players[0].position, players[1].position)))